In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zahidmughal2343/video-games-sale")

print("Path to dataset files:", path)

/home/userljt/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/userljt/.cache/kagglehub/datasets/zahidmughal2343/video-games-sale/versions/1


In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder.appName("Video Games Sale").getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/11 01:48:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import pandas as pd

data_path = '/home/userljt/.cache/kagglehub/datasets/zahidmughal2343/video-games-sale/versions/1'
sales = pd.read_csv(data_path+ '/video games sales.csv')
sales.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [4]:
# The highest sales in each year / platform / Genre
sales_highest_year = sales.groupby('Year')['Global_Sales'].max()
sales_highest_platform = sales.groupby('Platform')['Global_Sales'].max().sort_values(ascending=False)
sales_highest_genre = sales.groupby('Genre')['Global_Sales'].max().sort_values(ascending=False)
sales_highest_genre

Genre
Sports          82.74
Platform        40.24
Racing          35.82
Role-Playing    31.37
Puzzle          30.26
Misc            29.02
Shooter         28.31
Simulation      24.76
Action          21.40
Fighting        13.04
Adventure       11.18
Strategy         5.45
Name: Global_Sales, dtype: float64